In [ ]:
# takes 1h per 100k hands.
# Read acbl_hand_records.pkl file and clean up double dummy (DD) and par errors.
# Best to re-use previous cleaned up results as performance is slow. Use previous results unless record is new.
# It's not sufficient to validate par score vul calcs by simply correcting vul because some scores would be missing.
# Creates ../acbl/acbl_hand_records_cleaned.pkl

# todo:
# acbl_hand_records.pkl are incorrect due to legacy ignoring of vul. Clean here or correct sql in previous step? 
# figure out why this line throws an exception: stdhrs.at[grp_start+handno,'Par'] = pars
# board_record_string has replaced '10' with 'T'. This is confusing as it doesn't match sql. Needed?

# next steps:
# merge-hand-records.ipynb merges acbl and tcg standardized hand records?

# previous steps:
# acbl_sql_to_hand_records.ipynb creates acbl_hand_records.pkl

# Requirements
# https://github.com/Afwas/python-dds with some of my mods.

In [ ]:
import pandas as pd
import pathlib
import mlBridgeLib
from IPython.display import display # needed to define display() method in vscode

In [ ]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [ ]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [ ]:
# takes 20s
acbl_hand_records_filename = 'acbl_hand_records.pkl'
acbl_hand_records_file = acblPath.joinpath(acbl_hand_records_filename)
stdhrs = pd.read_pickle(acbl_hand_records_file)
display(len(stdhrs),stdhrs)

In [ ]:
# takes 20s + 1m + 1m
# similar to acbl_hand_records_dd_experiments
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
if acbl_hand_records_cleaned_file.exists():
    hrcdf = pd.read_pickle(acbl_hand_records_cleaned_file)
    display(len(hrcdf),hrcdf.head())
    # Preferred to use pd.merge but caused "out of memory" condition. Experiment with using dicts instead.
    DDmakesd = dict(zip(hrcdf['board_record_string'],hrcdf['DDmakes']))
    Pard = dict(zip(hrcdf['board_record_string'],hrcdf['Par']))
    #display(DDmakesd)
    #stdhrs = pd.merge(stdhrs,hrcdf[['EventBoard','DDmakes','Par']],on='EventBoard',how='left')
    #display(hrcdf[hrcdf['EventBoard'].isin(stdhrs['EventBoard'])]
    #display(len(stdhrs),stdhrs)
    # takes 1m
    DDmakesClean = stdhrs.apply(lambda r: DDmakesd[r['board_record_string']] if r['board_record_string'] in DDmakesd else None,axis='columns')
    b = DDmakesClean.notna() & stdhrs['DDmakes'].ne(DDmakesClean)
    assert all(~b)
    # takes 1m
    # verify that dirty Pars are still dirty
    ParClean = stdhrs.apply(lambda r: Pard[r['board_record_string']] if r['board_record_string'] in Pard else None,axis='columns')
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    display(stdhrs[b])
    # assign clean Pars. verify that all Pars are now cleaned.
    stdhrs.loc[b,'Par'] = ParClean
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    assert all(~b)
else:
    ParClean = pd.Series()

In [ ]:
# valdiate calculated dd result by comparing against known (assumed) correct dd. Similar to functions.CompareTable().
def CompareDDTables(DDtable1, DDtable2):
    for suit in range(dds.DDS_STRAINS):
        for pl in range(4):
            if DDtable1[suit][pl] != DDtable2[4 * suit + pl]:
                return False
    return True

In [ ]:
# valdiate calculated par result by comparing against known (assumed) correct par. Similar to functions.ComparePar().
def ComparePars(par1, par2):
    if par1[0] != par2[0]:
        return False
    if len(par1[1]) != len(par2[1]):
        return False
    for p1,p2 in zip(sorted(par1[1],key=lambda k: k[1]),sorted(par2[1],key=lambda k: k[1])):
        if p1 != p2:
            if p1[1] != p2[1] or p1[2] != p2[2] or p1[3] != p2[3]:
                return False # suit/double/direction difference
            if p1[0]+p1[4] != p2[0]+p2[4]:
                return False # only needs to have trick count same. ignore if levels are same (min level+overs vs max level+0)
    return True

In [ ]:
# undertricks: 0 = make; 1-13 = sacrifice
# overTricks: 0-3; e.g. 1 for 4S + 1
# level: 1-7
denom = 'NSHDC'
seats = ['N','E','S','W','NS','EW']

dealer_d = {'N':0, 'E':1, 'S':2, 'W':3}
vul_d = {'None':0, 'Both':1, 'N_S':2, 'E_W':3} # dds vul encoding is weird

In [ ]:
# perform initialization for calculation of double dummy (all positions and all strains) and Pars.

import dds
import ctypes
import functions

starting_handno = 0 # for restarting at a particular hando (0-len(stdhrs)). normally 0.

DDdealsPBN = dds.ddTableDealsPBN()
tableRes = dds.ddTablesRes()
pres = dds.allParResults()
presmaster = dds.parResultsMaster()

mode = 0
tFilter = ctypes.c_int * dds.DDS_STRAINS
trumpFilter = tFilter(0, 0, 0, 0, 0)
line = ctypes.create_string_buffer(80)

dds.SetMaxThreads(0)
max_tables = dds.MAXNOOFTABLES
df = stdhrs[ParClean.isna()]
nhands = len(df)
checkpoint_after_every = 1000

In [ ]:
# takes 1h per 100k hands.
# Perform dd and par calcs. Restartable.
# todo: implement direction in pbn. always using 'N:' for now.
# todo: some pars in acbl_hand_records.pkl are incorrect due to ignoring of vul. Need to correct pkl in previous step.

for n,grp_start in enumerate(range(starting_handno,nhands,max_tables)):
    n += 1
    grp_count = min(nhands-grp_start,max_tables)
    DDdealsPBN.noOfTables = grp_count
    print(f'Processing group:{n} hands:{grp_start} to {grp_start+grp_count} of {nhands}')
    DDtables = []
    for handno in range(grp_count):
        r = df.iloc[grp_start+handno]
        nsew_tuple = r['Hands']
        pbn = 'N:'+' '.join('.'.join([suit for suit in suits]) for suits in nsew_tuple)
        assert len(pbn) == 1+1+13*4+3*4+3, len(pbn) # 69=='N'+':'+(13 cards per hand)+(3 '.' per suit per hand)+3 spaces
        pbn_encoded = pbn.encode()
        #print(len(pbn),pbn,pbn_encoded)
        DDdealsPBN.deals[handno].cards = pbn_encoded
        nsew_tuple = r['DDmakes']
        DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
        assert len(DDtable) == 5*4, DDtable
        DDtables.append(DDtable)

    # CalcAllTablesPBN will do multi-threading
    res = dds.CalcAllTablesPBN(ctypes.pointer(DDdealsPBN), mode, trumpFilter, ctypes.pointer(tableRes), ctypes.pointer(pres))

    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, line)
        print(f'CalcAllTablesPBN: DDS error: {line.value.decode("utf-8")}')
        assert False, grp_start
    
    for handno in range(0, grp_count):
        r = df.iloc[grp_start+handno]
        #display(r)
        ref = r['ref'][1]
        dealer = r['Dealer']
        assert dealer in 'NESW'
        vul = r['Vul']
        assert vul in ['None','N_S','E_W','Both'], r
        par = r['Par']
        assert type(par) is tuple, r
        assert len(par) == 2, r
        assert type(par[0]) is int, r
        assert type(par[1]) is list, r
        assert len(par[1]) > 0, r
        par_result = ctypes.pointer(presmaster)
        dd_result = ctypes.pointer(tableRes.results[handno])

        # Par calculations are not multi-threading
        res = dds.DealerParBin(dd_result, par_result, dealer_d[dealer], vul_d[vul])
        if res != dds.RETURN_NO_FAULT:
            dds.ErrorMessage(res, line)
            print(f'DealerParBin: DDS error: {line.value.decode("utf-8")}')
            assert False, r
        
        score = par_result.contents.score
        if score == 0:
            pars = (0, [(0, '', '', '', 0)]) # par score is for everyone to pass (1 out of 100,000)
        else:
            assert par_result.contents.number > 0, r
            pars = (score,[])
            for i in range(par_result.contents.number):
                ct = par_result.contents.contracts[i]    
                #print(f'Par[{i}]: underTricks:{ct.underTricks} overTricks:{ct.overTricks} level:{ct.level} denom:{ct.denom} seats:{ct.seats}')
                assert ct.underTricks == 0 or ct.overTricks == 0
                pars[1].append((ct.level,denom[ct.denom],'*' if ct.underTricks else '',seats[ct.seats],ct.overTricks-ct.underTricks))

        dd_eq = CompareDDTables(dd_result.contents.resTable, DDtables[handno])
        par_eq = ComparePars(pars, par)
        if not dd_eq or not par_eq:
            functions.PrintPBNHand(f'Group:{n} Hand:{grp_start+handno+1}/{nhands} ACBL Ref:{ref} Vul:{vul}', DDdealsPBN.deals[handno].cards)
            functions.PrintTable(dd_result)
            if not dd_eq:
                print(f'DD mismatch: Current:{DDtables[handno]}')
                # DDtable = dd_result.contents.resTable
                #DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
                # stdhrs.at[grp_start+handno,'DDmakes'] = DDtable
                assert False # temp - need to finish compatible DDtable
            if not par_eq:
                #functions.PrintDealerParBin(par_result)
                if pars[0] != par[0]: # differences in par score are usually due to vul-unknown vs vul-known.
                    print(f'Par score mismatch: Vul ignorance yielding wrong score?: Calculated:{pars[0]}, Current:{par[0]}')
                else:
                    print(f'Par contract mismatch: (Vul ignorance effecting interfering bid?): Calculated:{pars[1]}, Current:{par[1]}')
                    #assert False
                stdhrs.at[r.name,'Par'] = pars
            print()
    if n%checkpoint_after_every == 0: # save every checkpoint_after_every count (1000).
        acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
        acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
        stdhrs.to_pickle(acbl_hand_records_cleaned_file) # takes 40s
        print(f'Checkpoint at group {n}: df written to {acbl_hand_records_cleaned_file}')
        starting_handno = grp_start
starting_handno = nhands

In [ ]:
# takes 40s
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
stdhrs.to_pickle(acbl_hand_records_cleaned_file)